In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU

Reading Dataset

In [ ]:
ds = pd.read_csv('/content/drive/MyDrive/EFQM/5-Multi label classification/Multivariate_pollution.csv')
ds.head()

Check target values

In [ ]:
print('snow data', ds['snow'].unique())
print('rain data', ds['rain'].unique())

Checking all data

In [ ]:
ds.info()

Creating time features

In [ ]:
ds['date'] = pd.to_datetime(ds['date'])
ds['year'] = ds['date'].dt.year
ds['month'] = ds['date'].dt.month
ds['day'] = ds['date'].dt.day
ds['hour'] = ds['date'].dt.hour


ds.drop(['date'], axis=1, inplace=True)
ds.head()

create dummy data from wind_dir feature

In [ ]:
dummy_wind = pd.get_dummies(ds['wnd_dir'])
data = pd.concat([ds, dummy_wind], axis=1)
ds.drop(['wnd_dir'], axis=1, inplace=True)
ds.head()

Train, Test, and Validation set splitting

In [ ]:
from sklearn.model_selection import train_test_split

X = ds.drop(['snow', 'rain'], axis=1)
y = ds[['snow', 'rain']]

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
X_train.shape, X_val.shape, X_test.shape

Feature scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_val = pd.DataFrame(scaler.fit_transform(X_val), columns=X_val.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)

X_train.head()

Model

In [ ]:
model = tf.keras.Sequential([
    Dense(64, input_shape=(X_train.shape[1],)),
    LeakyReLU(alpha=0.01),

    Dense(32),
    LeakyReLU(alpha=0.01),

    Dense(16),
    LeakyReLU(alpha=0.01),

    Dense(8),
    LeakyReLU(alpha=0.01),

    Dense(4),
    LeakyReLU(alpha=0.01),

    Dense(2),
    Dropout(0.5),
])

model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X_train, y_train, epochs=128, batch_size=1028, validation_data=(X_val, y_val),
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=32)])

Plot Training Results

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

Model Evaluation

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(X_test)
print(mean_squared_error(y_test, y_pred))
print(r2_score(y_test, y_pred))